In [ ]:
!pip install deep_sort_realtime
!pip install ultralytics
!pip install opencv-python


In [ ]:
from ultralytics import YOLO
import cv2

# تحميل النموذج المدرب باستخدام YOLO
model = YOLO('best2.pt')  # تأكد من تعديل المسار إلى المسار الصحيح للنموذج

# فتح الكاميرا
cap = cv2.VideoCapture(0)  # استخدام الكاميرا الافتراضية (0)

while True:
    # قراءة الصورة من الكاميرا
    ret, frame = cap.read()

    if not ret:
        print("فشل في قراءة الإطار من الكاميرا.")
        break
    
    # إجراء الكشف باستخدام النموذج المدرب
    results = model(frame)

    # استخراج النتائج من النموذج
    # عادة النتائج تحتوي على بعض الخصائص مثل boxes, labels, and confidences
    boxes = results[0].boxes.xywh.cpu().numpy()  # إحداثيات الإطارات (boxes)
    confidences = results[0].boxes.conf.cpu().numpy()  # الثقة
    labels = results[0].boxes.cls.cpu().numpy()  # الفئات (التسميات)

    # أسماء الفئات (يجب التأكد من أنها تتطابق مع أسماء الحلوى التي دربها النموذج)
    class_names = results[0].names  # أسماء الفئات التي تم التدريب عليها

    for i in range(len(boxes)):
        # إذا كانت الثقة أكبر من 50%
        if confidences[i] > 0.5:
            x1, y1, w, h = boxes[i]
            x1, y1, x2, y2 = int(x1 - w / 2), int(y1 - h / 2), int(x1 + w / 2), int(y1 + h / 2)
            
            # استخراج التسمية (الفئة)
            label = class_names[int(labels[i])]  # استرجاع الاسم الصحيح للفئة
            confidence = confidences[i]
            
            # رسم المستطيل حول الكائن المكتشف
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # عرض النتيجة
    cv2.imshow('Candy Detection', frame)
    
    # الخروج من الحلقة عند الضغط على مفتاح 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# إغلاق الكاميرا
cap.release()
cv2.destroyAllWindows()
